In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from __future__ import print_function, division

from sympy import S, Expr, Mul,Add, Pow, symbols, Function, Derivative, simplify, sympify
from sympy.physics.quantum import Operator
from sympy.printing.latex import print_latex
from sympy.printing.pretty.stringpict import prettyForm


from sympy import S, Expr, Mul, Add, Pow, Rational
from sympy.printing.pretty.stringpict import prettyForm

from sympy.physics.quantum.dagger import Dagger
from sympy.physics.quantum.operator import Operator

from sympy.core.decorators import _sympifyit, call_highest_priority
from sympy.core.function import UndefinedFunction

from termcolor import colored

In [3]:
A = Function("A", commutative=False)
B = Function("B", commutative=False)
C = Function("C", commutative=False)
D = Function("D", commutative=False)

x, px, y, py = symbols("x px y py")

half = Rational(1, 2)

In [4]:
def debugprint(obj):
    try:
        print(obj.args)
    except:
        pass
    try:
        print(vars(obj.args[1]))
    except:
        pass

In [5]:
def orderArgs(cls, A,B):
    if isinstance(A, UndefinedFunction):
        tmp = symbols("tmp")
        if isinstance(B, UndefinedFunction):
            if A(tmp).compare(B(tmp)) ==1:
                  return S.NegativeOne * cls(B,A)
        elif A(tmp).compare(B) == 1:
            print("reversing - undef - def")
            return S.NegativeOne * cls(B,A)
    elif isinstance(B, UndefinedFunction):
        tmp = symbols("tmp")
        if A.compare(B(tmp)) ==1:
            print("reversing - def - undef")
            return S.NegativeOne * cls(B,A) 
    else:
        if A.compare(B) ==1:
            print("reversing - def - def")

            return S.NegativeOne * cls(B,A) 
        return None

In [12]:
class PoissonBracket(Expr):
    _op_priority = 11.0
    _is_commutative=False
    
    def __new__(cls,f,g,**kwargs):
        A = sympify(f)
        B = sympify(g)
        
        if A==B:
            return S.Zero
        elif A.is_number or B.is_number:
            return S.Zero
                
        obj = orderArgs(cls,A,B)
        
        if obj is not None:
#             debugprint(obj)
            obj.args[1].coords = kwargs.get("coords",None)
            obj.args[1].mom = kwargs.get("mom",None)
            return obj
        else:
            obj = Expr.__new__(cls,A,B)
            obj.A = A
            obj.B = B
#             debugprint(obj)
            return obj
    
    def __init__(self, f,g,**kwargs):
        self.coords = kwargs.get("coords",None)
        self.mom = kwargs.get("mom",None)
        
    def _eval_derivative(self, symbol):
        new_expr = self.doit().diff(symbol)
        return new_expr
           
    def doit(self, **hints):
#         if isinstance(self,Mul):
#             if sympify(self.args[0]) == S.NegativeOne:
#                 c = self.args[1].coords
#                 m = self.args[1].mom
#         else:
        c = self.coords
        m = self.mom
        if (m is None or c is None):
            print(colored("Nothing to evaluate - missing coordinates and momenta!","red"))
            return self
        elif len(c) != len(m):
            print(colored("Nothing to evaluate - uneven number of coordinates and momenta!","red"))
            return self
        A = self.args[0]
        B = self.args[1]
    
        
        # if one of the args is a Poisson Bracket 
        # the outer c and m need to be used
        # we therefore overwrite them
        if isinstance(A,PoissonBracket):
            A.coords = c
            A.mom = m
        if isinstance(B,PoissonBracket):
            B.coords = c
            B.mom = m
            
        
#         print(vars(A))
#         print(vars(B))
        
        hp = S(0)
        for qi,pi in zip(c,m):
            hp += A.diff(qi) * B.diff(pi)
            hp -= A.diff(pi) * B.diff(qi)
        return hp
    
        

In [13]:
import ipytest
import pytest
ipytest.autoconfig()

In [14]:
%%run_pytest[clean]

single_pb_pass = [
    (         (S.One   , half) , {}            , 0 ), 
    (         (A       , half) , {}            , 0 ),
    (         (A(x)    , half) , {}            , 0 ),
    (         (A(x,px) , half) , {}            , 0 ),
    (         (S.One   , half) , {"coords":[x]}, 0 ),
    (         (A       , half) , {"coords":[x]}, 0 ),
    (         (A(x)    , half) , {"coords":[x]}, 0 ),
    (         (A(x,px) , half) , {"coords":[x]}, 0 ),
    (reversed((S.One   , half)), {}            , 0 ),
    (reversed((A       , half)), {}            , 0 ),
    (reversed((A(x)    , half)), {}            , 0 ),
    (reversed((A(x,px) , half)), {}            , 0 ),
    (reversed((S.One   , half)), {"coords":[x]}, 0 ),
    (reversed((A       , half)), {"coords":[x]}, 0 ),
    (reversed((A(x)    , half)), {"coords":[x]}, 0 ),
    (reversed((A(x,px) , half)), {"coords":[x]}, 0 ),
]

# create dict to lookup test in list if one fails
single_pb_pass_dict = dict(zip(range(1,len(single_pb_pass)+1), single_pb_pass))

@pytest.mark.parametrize('input,kwargs,expected',single_pb_pass)
def test_single_pb_parametrized_pass(input, kwargs, expected):
    test = PoissonBracket(*input,**kwargs)
    assert test == expected

................                       [100%]
16 passed in 0.06s


In [15]:
%%run_pytest[clean]

single_pb_doit_pass = [
    ( (A       , B       )            , {}            , PoissonBracket(A, B)             ),
    ( (A(x,px) , B       )            , {}            , PoissonBracket(A(x,px), B)       ),
    ( (A       , B(x,px) )            , {}            , PoissonBracket(A, B(x,px))       ),
    ( (A(x,px) , B(x,px) )            , {}            , PoissonBracket(A(x,px), B(x,px)) ),
    ( (x **2   , x + px  )            , {}            , PoissonBracket(x ** 2, x + px)   ),
    
    ( reversed((A       , B       )), {}            , PoissonBracket(B, A)             ),
    ( reversed((A(x,px) , B       )), {}            , PoissonBracket(B, A(x,px))       ),
    ( reversed((A       , B(x,px) )), {}            , PoissonBracket(B(x,px),A)        ),
    ( reversed((A(x,px) , B(x,px) )), {}            , PoissonBracket(B(x,px),A(x,px))  ),
    
    ( (A       , B       )            , {"coords":[x]}, PoissonBracket(A, B)),
    ( (A(x,px) , B       )            , {"coords":[x]}, PoissonBracket(A(x,px), B)),
    ( (A       , B(x,px) )            , {"coords":[x]}, PoissonBracket(A, B(x,px))),
    ( (A(x,px) , B(x,px) )            , {"coords":[x]}, PoissonBracket(A(x,px), B(x,px))),
    
    (
        (x **2     , x + px     ),
        {"coords":[x]  , "mom": [px]   },
        2*x
    ),
    (
        (A(x, px)  , B(x, y, py)), 
        {"coords":[x]  , "mom": [px]   }, 
        -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)
    ),
    (
        (A(x,y, px), B(x, y, py)),
        {"coords":[x]  , "mom": [px]   },
        -Derivative(A(x, y, px), px)*Derivative(B(x, y, py), x)
    ),
    (
        (A(x, px)  , B(x, y, py)),
        {"coords":[x,y], "mom": [px]   }, 
        PoissonBracket(A(x, px), B(x, y, py))
    ),
    (
        (A(x, px)  , B(x, y, py)), 
        {"coords":[x,y], "mom": [px,py]}, 
        -Derivative(A(x, px), px)*Derivative(B(x, y, py), x)
    ),
    (
        (A(x,y, px), B(x, y, py)), 
        {"coords":[x,y], "mom": [px,py]}, 
        -Derivative(A(x, y, px), px)*Derivative(B(x, y, py), x) +Derivative(A(x, y, px), y)*Derivative(B(x, y, py), py)
    ),
    
    (
        (A(x,px)+B(x,y,py), C(x,px)           ),
        {"coords":[x,y], "mom":[px,py]}, 
        (Derivative(A(x, px), x) + Derivative(B(x, y, py), x))*Derivative(C(x, px), px) - Derivative(A(x, px), px)*Derivative(C(x, px), x) 
    ),
    (
        (A(x,px)*B(x,y,py), C(x,px)           ), 
        {"coords":[x,y], "mom":[px,py]}, 
        (A(x, px)*Derivative(B(x, y, py), x) + Derivative(A(x, px), x)*B(x, y, py))*Derivative(C(x, px), px) - Derivative(A(x, px), px)*B(x, y, py)*Derivative(C(x, px), x)
    ),
    (
        (A(x,px)           , B(x,y,py)*C(x,px)), 
        {"coords":[x,y], "mom":[px,py]}, 
        S.NegativeOne * ((B(x, y, py)*Derivative(C(x, px), x) + Derivative(B(x, y, py), x)*C(x, px))*Derivative(A(x, px), px) - B(x, y, py)*Derivative(C(x, px), px)*Derivative(A(x, px), x))
    ),
    (
        (A(x,px)**3        , B(x,y,py)        ), 
        {"coords":[x,y], "mom":[px,py]}, 
        -3*A(x, px)**3*Derivative(A(x, px), px)*A(x, px)**(-1)*Derivative(B(x, y, py), x)
    ),
]

# create dict to lookup test in list if one fails
single_pb_doit_pass_dict = dict(zip(range(1,len(single_pb_doit_pass)+1), single_pb_doit_pass))

@pytest.mark.parametrize('input,kwargs,expected',single_pb_doit_pass)
def test_single_pb_doit_parametrized_pass(input, kwargs, expected):
    test = PoissonBracket(*input,**kwargs)
    assert test.doit() == expected

reversing - undef - def
reversing - def - undef
reversing - def - def
.......................                [100%]
23 passed in 0.11s


In [16]:
%%run_pytest[clean]

double_pb_pass = [
    ((S.One   , half), (S.One,) ,{}            , {}, 0),
    ((A       , half), (S.One,) ,{}            , {}, 0),
    ((A(x)    , half), (S.One,) ,{}            , {}, 0),
    ((A(x,px) , half), (S.One,) ,{}            , {}, 0),
    ((S.One   , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A       , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (S.One,) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (S.One,) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A,) ,{}            , {}, 0),
    ((A       , half), (A,) ,{}            , {}, 0),
    ((A(x)    , half), (A,) ,{}            , {}, 0),
    ((A(x,px) , half), (A,) ,{}            , {}, 0),
    ((S.One   , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A,) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A,) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A(x),) ,{}            , {}, 0),
    ((A       , half), (A(x),) ,{}            , {}, 0),
    ((A(x)    , half), (A(x),) ,{}            , {}, 0),
    ((A(x,px) , half), (A(x),) ,{}            , {}, 0),
    ((S.One   , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A(x),) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A(x),) ,{"coords":[x]}, {}, 0),
    
    ((S.One   , half), (A(x,px),) ,{}            , {}, 0),
    ((A       , half), (A(x,px),) ,{}            , {}, 0),
    ((A(x)    , half), (A(x,px),) ,{}            , {}, 0),
    ((A(x,px) , half), (A(x,px),) ,{}            , {}, 0),
    ((S.One   , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A       , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A(x)    , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    ((A(x,px) , half), (A(x,px),) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A       , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (S.One,) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (S.One,) ,{}            , {}, 0),
    (reversed((S.One   , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (S.One,) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A,) ,{}            , {}, 0),
    (reversed((A       , half)), (A,) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A,) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A,) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A,) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A,) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A       , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A(x),) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A(x),) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A(x),) ,{"coords":[x]}, {}, 0),
    
    (reversed((S.One   , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A       , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A(x)    , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((A(x,px) , half)), (A(x,px),) ,{}            , {}, 0),
    (reversed((S.One   , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A       , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x)    , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    (reversed((A(x,px) , half)), (A(x,px),) ,{"coords":[x]}, {}, 0),
    
    ((A     , B    ), (S.One,) ,{}            , {}, 0),
]

@pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_pass)
def test_double_pb_parametrized_pass(input1, input2, kwargs1, kwargs2, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    assert pb2 == expected

...................................... [ 58%]
...........................            [100%]
65 passed in 0.22s


In [17]:
%%run_pytest[clean]


double_pb_doit_pass = [
    ((A      , B      ), (S.One,), {}, {}, 0),
    ((A      , B      ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A,B),C)),
    ((A(x,px), B(x,px)), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C)),
    ((x **2  , x + px ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(x**2,x+px),C)),
    
    ((A      , B      ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A, B),C)),
    ((A(x)   , B      ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A(x), B), C)),
    ((A      , B(x)   ), (C    ,), {}, {}, PoissonBracket(PoissonBracket(A, B(x)), C)),
    ((A      , B      ), (C(x) ,), {}, {}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    ((A      , B      ), (C    ,), {"coords":[x]}, {}, PoissonBracket(PoissonBracket(A, B),C)),
    ((A(x)   , B      ), (C    ,), {"coords":[x]}, {}, PoissonBracket(PoissonBracket(A(x), B), C)),
    ((A      , B(x)   ), (C    ,), {"coords":[x]}, {}, PoissonBracket(PoissonBracket(A, B(x)), C)),
    ((A      , B      ), (C(x) ,), {"coords":[x]}, {}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    
    ((A(x)  , B       ), (C    ,), {}, {"coords":[x]}, PoissonBracket(PoissonBracket(A(x), B), C)),
    ((A     , B(x)    ), (C    ,), {}, {"coords":[x]}, PoissonBracket( PoissonBracket(A, B(x)),C )),
    ((A     , B       ), (C(x) ,), {}, {"coords":[x]}, S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A, B))),
    ((A     , B       ), (C    ,), {}, {"coords":[x]}, PoissonBracket(PoissonBracket(A, B), C)),
   
    ((A(x)   , B(x)   ), (C      ,), {"coords":[x]}, {}            , PoissonBracket(PoissonBracket(A(x), B(x)), C)),
    ((A(x)   , B(x)   ), (C(x)   ,), {"coords":[x]}, {}            , S.NegativeOne * PoissonBracket(C(x), PoissonBracket(A(x), B(x)))),
    ((A(x)   , B(x)   ), (C(x)   ,), {}            , {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x]}, {}            , -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {}, {}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),

    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x], "mom": [px]}, {}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {}, {"coords":[x], "mom": [px]}, Derivative(C(x), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)),
    ((A(x)   , B(x,px)), (C(x)   ,), {}, {"coords":[x], "mom": [px]}, -Derivative(C(x), x)*Derivative(A(x), x)*Derivative(B(x, px), (px, 2))),
    ((A(x)   , B(x)   ), (C(x,px),), {}, {"coords":[x], "mom": [px]}, 0),
    ((A(x,px), B(x,px)), (C(x)   ,), {}, {"coords":[x], "mom": [px]}, -Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))),
    ((A(x,px), B(x)   ), (C(x,px),), {}, {"coords":[x], "mom": [px]}, S.NegativeOne * (-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x))),
    ((A(x,px), B(x,px)), (C(x,px),), {}, {"coords":[x], "mom": [px]}, S.NegativeOne *(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x)))),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x), B(x, px)))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x), B(x)))),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x), PoissonBracket(A(x, px), B(x,px)))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x)))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x]}, -PoissonBracket(C(x,px), PoissonBracket(A(x, px), B(x,px)))),
    
    ((A(x,px), B(x)   ), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, Derivative(C(x), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x)),
    ((A(x)   , B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, -Derivative(C(x), x)*Derivative(A(x), x)*Derivative(B(x, px), (px, 2))),
    ((A(x)   , B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, 0),
    ((A(x,px), B(x,px)), (C(x)   ,), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, -Derivative(C(x), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))),
    ((A(x,px), B(x)   ), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]},  S.NegativeOne * (-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x), (x, 2)) - Derivative(A(x, px), px, x)*Derivative(B(x), x)) - Derivative(C(x, px), x)*Derivative(A(x, px), (px, 2))*Derivative(B(x), x))),
    ((A(x,px), B(x,px)), (C(x,px),), {"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]}, S.NegativeOne* (-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))),
    
    ((A(x,px),B),(C,), {}, {}, PoissonBracket(PoissonBracket(A(x, px), B), C)),
    ((x **2  , x + px ), (C       ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(x**2,x+px,coords=[x],mom=[px]),C)),
    ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C(x,px))),
    ((A(x,px), B(x,px)), (C       ,) ,{"coords":[x], "mom": [px]}, {}, PoissonBracket(PoissonBracket(A(x,px),B(x,px)),C)),
    
    ((A(x,px), B(x,px)), (C(x,px) ,) ,{"coords":[x], "mom": [px]}, {"coords":[x], "mom": [px]},S.NegativeOne *(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px)))),
    
]

# create dict to lookup test in list if one fails
double_pb_doit_pass_dict = dict(zip(range(1,len(double_pb_doit_pass)+1), double_pb_doit_pass))

@pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_doit_pass)
def test_double_pb_doit_parametrized_pass(input1, input2, kwargs1, kwargs2, expected):
    pb1 = PoissonBracket(*input1, **kwargs1)
    pb2 = PoissonBracket(pb1, *input2, **kwargs2)
    print(pb2.doit()-expected)
    assert pb2.doit() == expected

# @pytest.mark.parametrize('input1,input2,kwargs1,kwargs2,expected',double_pb_doit_fail)
# def test_double_pb_doit_parametrized_fail(input1, input2, kwargs1, kwargs2, expected):
#     with pytest.raises(AttributeError):
#         pb1 = PoissonBracket(*input1, **kwargs1)
#         pb2 = PoissonBracket(pb1, *input2, **kwargs2)

reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - undef
reversing - def - def
reversing - def - undef
...................................... [ 56%]
.............................          [100%]
67 passed in 0.39s


In [87]:
input1, input2, kwargs1,kwargs2, expected = double_pb_doit_pass_dict.get(50)
print(input1, input2, kwargs1, kwargs2)
pb1 = PoissonBracket(*input1, **kwargs1)
pb2 = PoissonBracket(pb1, *input2, **kwargs2)
pb2.doit(), expected

(A(x, px), B(x, px)) (C(x, px),) {} {'coords': [x], 'mom': [px]}
reversing - def - def


(-(-Derivative(C(x, px), px)*(-Derivative(A(x, px), px)*Derivative(B(x, px), (x, 2)) + Derivative(A(x, px), x)*Derivative(B(x, px), px, x) + Derivative(A(x, px), (x, 2))*Derivative(B(x, px), px) - Derivative(A(x, px), px, x)*Derivative(B(x, px), x)) + Derivative(C(x, px), x)*(-Derivative(A(x, px), px)*Derivative(B(x, px), px, x) - Derivative(A(x, px), (px, 2))*Derivative(B(x, px), x) + Derivative(A(x, px), x)*Derivative(B(x, px), (px, 2)) + Derivative(A(x, px), px, x)*Derivative(B(x, px), px))),
 -PoissonBracket(C(x, px), PoissonBracket(A(x, px), B(x, px))))

In [48]:
input, kwargs, expected = single_pb_doit_pass_dict.get(22)
print(input,kwargs)
test = PoissonBracket(*input,**kwargs)
test.doit(),

(A(x, px), B(x, y, py)*C(x, px)) {'coords': [x, y], 'mom': [px, py]}
reversing - def - def
(-1, PoissonBracket(B(x, y, py)*C(x, px), A(x, px)))
{'A': B(x, y, py)*C(x, px), 'B': A(x, px), 'coords': None, 'mom': None}
Nothing to evaluate - missing coordinates and momenta!


(-PoissonBracket(B(x, y, py)*C(x, px), A(x, px)),)